In [1]:
!pip3 install torch torchvision torchaudio torchmetrics




[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: C:\Users\pysol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!pip uninstall transformers
!pip install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/6a/dc/23c26b7b0bce5aaccf2b767db3e9c4f5ae4331bd47688c1f2ef091b23696/transformers-4.42.4-py3-none-any.whl.metadata
  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\pysol\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\transformers\\models\\deprecated\\trajectory_transformer\\convert_trajectory_transformer_original_pytorch_checkpoint_to_pytorch.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: C:\Users\pysol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import transformers

In [4]:
# https://lightning.ai/docs/torchmetrics/stable/multimodal/clip_iqa.html
import torch
from torchmetrics.multimodal.clip_iqa import CLIPImageQualityAssessment
from torchvision import transforms
from PIL import Image

"""
working
torch==2.3.1
torchvision==0.18.1
torchmetrics==1.4.0.post0 
huggingface-hub==0.24.0
transformers==4.42.4


not working
transformers==4.43.1
huggingface-hub==0.24.1
torch==2.3.1
torchaudio==2.3.1
torchmetrics==1.4.0.post0
torchvision==0.18.1
"""

def load_image(
    image_path:str,
    transforms:transforms.Compose
)->torch.tensor:
    # Open the image file
    img = Image.open(image_path).convert('RGB')  # Convert to RGB
    # display(img)
    # display(img.size)
    img_tensor = transforms(img)  # Apply the transformation
    return img_tensor.unsqueeze(0)

path = 'sample_images/Complex Imaging Tags _ Metal Marker Mfg.jpeg'
_ = torch.manual_seed(42)
# Define a transform to convert the image to a tensor and normalize it
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize for pre-trained models
                         std=[0.229, 0.224, 0.225])
])
img = load_image(path,transform)
metric = CLIPImageQualityAssessment(
    # model_name_or_path='openai/clip-vit-base-patch16',
    prompts=(("Good Photo.", "Bad Photo."), "quality")
)
metric(img)
# /home/pysolver33/miniconda3/envs/vqaapi/bin/python

{'user_defined_0': tensor(0.8896), 'quality': tensor(0.8896)}

In [2]:
path = 'sample_images\Complex Imaging Tags _ Metal Marker Mfg (1).jpeg'
img = load_image(path,transform)
metric = CLIPImageQualityAssessment(prompts=(("Good Photo.", "Bad Photo."), "quality"))
metric(img)

{'user_defined_0': tensor(0.7124), 'quality': tensor(0.7124)}

In [3]:
path = 'sample_images\image (6).png'
img = load_image(path,transform)
metric = CLIPImageQualityAssessment(prompts=(("Good Photo.", "Bad Photo."), "quality"))
metric(img)

{'user_defined_0': tensor(0.9425), 'quality': tensor(0.9425)}

In [4]:
path = 'sample_images\IMG_9821.jpeg'
img = load_image(path,transform)
metric = CLIPImageQualityAssessment(prompts=(("Good Photo.", "Bad Photo."), "quality"))
metric(img)

{'user_defined_0': tensor(0.6709), 'quality': tensor(0.6709)}

# APP

In [1]:
import openai
from openai import OpenAI
import uvicorn
import base64
import requests
import json
from fastapi import FastAPI, Request, Form,UploadFile,File
from fastapi.responses import HTMLResponse, RedirectResponse,JSONResponse,RedirectResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles

from pydantic_settings import BaseSettings
import io
import torch
from PIL import Image
from torchvision import transforms
from torchmetrics.multimodal.clip_iqa import CLIPImageQualityAssessment


class Settings(BaseSettings):
    OPENAI_API_KEY: str = 'OPENAI_API_KEY'
    FLASK_APP: str = 'FLASK_APP'
    FLASK_ENV: str = 'FLASK_ENV'
    
    class Config:
        env_file = '.env'

settings = Settings()
client = OpenAI(api_key=settings.OPENAI_API_KEY)
app = FastAPI()

templates = Jinja2Templates(directory="templates")
app.mount("/static", StaticFiles(directory="static"), name="static")

# Function to encode the image
def encode_image(image_path:str)->base64:
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_json(
    img_bytes:base64
)->dict:
    # Getting the base64 string
    # base64_image = encode_image(file_name)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {settings.OPENAI_API_KEY}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": "You are an intelligent system api that reads the texts from an image and outputs the texts as key values pairs in JSON format. Given an image, output the texts in JSON format."
                },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{img_bytes}"
                        }
                }
            ]
        }
    ],
        "max_tokens": 1000
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        # Save the response to a file
        with open("response_data.json", "w") as file:
            file.write(response.text)
    return response.json()
    # return templates.TemplateResponse("json_out.html", {"request": request, "data": response.json()})


@app.get("/", response_class=HTMLResponse)
def index(request: Request):
    return templates.TemplateResponse("upload_image.html", {"request": request})


@app.post("/output_iqa")
async def create_upload_file(
    request:Request,
    file: UploadFile = File(...),
    choice: str = Form(...), 
)->dict:
    pairs = {
        'quality': ("Good photo", "Bad photo"),
        'sharpness': ("Sharp photo", "Blurry photo"),
        'noisiness': ("Clean photo", "Noisy photo"),
    }
    classes = pairs[choice.lower()]
    
    _ = torch.manual_seed(42)
    transform = transforms.Compose([
        transforms.ToTensor(),  # Converts the image to a PyTorch tensor
        # transforms.Normalize(mean=0,std=255)
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize for pre-trained models
                            std=[0.229, 0.224, 0.225])
    ])
    
    # Read the image file into a PIL Image
    image_content = await file.read()
    image = Image.open(io.BytesIO(image_content)).convert('RGB')
    img = transform(image)
    metric = CLIPImageQualityAssessment(
        # model_name_or_path='openai/clip-vit-large-patch14',
        prompts=(classes, choice)
    )
    out = {
        'filename':file.filename,
        'on':choice,
        "prompts":classes,
        "score": metric(img)[choice].item(),
    }
    if out['score'] > 0.75:
        # return RedirectResponse(url=f'/get_json/?filename={file.filename}')
        to_gpt = base64.b64encode(image.tobytes()).decode('utf-8')
        info = get_json(to_gpt)
        out.update(json.loads(info['choices'][0]['message']['content']))
        return out
    return out


if __name__ == "__main__":
    import logging
    # NEED BELOW
    # https://stackoverflow.com/questions/69774921/im-using-wsl-how-i-upgrade-python-to-the-last-version-through-the-console
    # Disable uvicorn access logger
    uvicorn_access = logging.getLogger("uvicorn.access")
    uvicorn_access.disabled = True

    logger = logging.getLogger("uvicorn")
    logger.setLevel(logging.getLevelName(logging.DEBUG))
    uvicorn.run('app:app', host="localhost", port=5001, reload=True)


INFO:     Will watch for changes in these directories: ['c:\\Users\\pysol\\Desktop\\projects\\openai-quickstart-fastapi']
INFO:     Uvicorn running on http://localhost:5001 (Press CTRL+C to quit)
INFO:     Started reloader process [9444] using WatchFiles
